In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
cols = ['status', 'duration', 'credit_hist', 'purpose', 'credit_amt', 'savings', 'employment', 'installment_rate', 'personal_status', 'debtors', 'residencesince', 'property', 'age', 'install_plans', 'housing', 'existing_credits', 'job', 'maintenance_paying_people', 'telephone', 'foreign_worker', 'result']



In [3]:
df = pd.read_table('german.data', names=cols, sep=" ", index_col=False)

In [4]:
df.head()

,status,duration,credit_hist,purpose,credit_amt,savings,employment,installment_rate,personal_status,debtors,...,property,age,install_plans,housing,existing_credits,job,maintenance_paying_people,telephone,foreign_worker,result
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,2


In [5]:
def preprocess_german(df): 
    df['status'] = df['status'].map({'A11': 0, 'A12': 1, 'A13': 2, 'A14': 3}).astype(int)
    df['credit_hist'] = df['credit_hist'].map({'A34': 0, 'A33': 1, 'A32': 2, 'A31': 3, 'A30': 4}).astype(int)
    df.loc[(df['credit_amt'] <= 2000), 'credit_amt'] = 0
    df.loc[(df['credit_amt'] > 2000) & (df['credit_amt'] <= 5000), 'credit_amt'] = 1
    df.loc[(df['credit_amt'] > 5000), 'credit_amt'] = 2   
    df.loc[(df['duration'] <= 12), 'duration'] = 0
    df.loc[(df['duration'] > 12) & (df['duration'] <= 24), 'duration'] = 1
    df.loc[(df['duration'] > 24) & (df['duration'] <= 36), 'duration'] = 2
    df.loc[(df['duration'] > 36), 'duration'] = 3
    df['age'] = df['age'].apply(lambda x : 1 if x >= 45 else 0) # 1 if old, 0 if young
    df['savings'] = df['savings'].map({'A61': 0, 'A62': 1, 'A63': 2, 'A64': 3, 'A65': 4}).astype(int)
    df['employment'] = df['employment'].map({'A71': 0, 'A72': 1, 'A73': 2, 'A74': 3, 'A75': 4}).astype(int)    
    df['gender'] = df['personal_status'].map({'A91': 1, 'A92': 0, 'A93': 1, 'A94': 1, 'A95': 0}).astype(int)
    df['debtors'] = df['debtors'].map({'A101': 0, 'A102': 1, 'A103': 2}).astype(int)
    df['property'] = df['property'].map({'A121': 3, 'A122': 2, 'A123': 1, 'A124': 0}).astype(int)        
    df['install_plans'] = df['install_plans'].map({'A141': 1, 'A142': 1, 'A143': 0}).astype(int)
    df['job'] = df['job'].map({'A171': 0, 'A172': 1, 'A173': 2, 'A174': 3}).astype(int)    
    df['telephone'] = df['telephone'].map({'A191': 0, 'A192': 1}).astype(int)
    df['foreign_worker'] = df['foreign_worker'].map({'A201': 1, 'A202': 0}).astype(int)
    pd.get_dummies(df, columns=['purpose', 'housing'], drop_first=True)
    
    return df



In [6]:
X = preprocess_german(df);

In [7]:
df.head()

,status,duration,credit_hist,purpose,credit_amt,savings,employment,installment_rate,personal_status,debtors,...,age,install_plans,housing,existing_credits,job,maintenance_paying_people,telephone,foreign_worker,result,gender
0,0,0,0,A43,0,4,4,4,A93,0,...,1,0,A152,2,2,1,1,1,1,1
1,1,3,2,A43,2,0,2,2,A92,0,...,0,0,A152,1,2,1,0,1,2,0
2,3,0,0,A46,1,0,3,2,A93,0,...,1,0,A152,1,1,2,0,1,1,1
3,0,3,2,A42,2,0,3,2,A93,2,...,1,0,A153,1,2,2,0,1,1,1
4,0,1,1,A40,1,0,2,3,A93,0,...,1,0,A153,2,2,2,0,1,2,1


In [8]:
df = df.drop(["purpose", "housing"], axis=1)

In [9]:
df = df.drop(["personal_status"], axis=1)

In [10]:
df.columns

Index(['status', 'duration', 'credit_hist', 'credit_amt', 'savings',
       'employment', 'installment_rate', 'debtors', 'residencesince',
       'property', 'age', 'install_plans', 'existing_credits', 'job',
       'maintenance_paying_people', 'telephone', 'foreign_worker', 'result',
       'gender'],
      dtype='object')

In [11]:
X = df[['status', 'duration', 'credit_hist', 'credit_amt', 'savings',
       'employment', 'installment_rate', 'debtors', 'residencesince',
       'property', 'age', 'install_plans', 'existing_credits', 'job',
       'maintenance_paying_people', 'telephone', 'foreign_worker',
       'gender']]

In [12]:
y = df[["result"]]

In [13]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=15)

In [14]:
x_train.columns

Index(['status', 'duration', 'credit_hist', 'credit_amt', 'savings',
       'employment', 'installment_rate', 'debtors', 'residencesince',
       'property', 'age', 'install_plans', 'existing_credits', 'job',
       'maintenance_paying_people', 'telephone', 'foreign_worker', 'gender'],
      dtype='object')

In [15]:
df.head()


,status,duration,credit_hist,credit_amt,savings,employment,installment_rate,debtors,residencesince,property,age,install_plans,existing_credits,job,maintenance_paying_people,telephone,foreign_worker,result,gender
0,0,0,0,0,4,4,4,0,4,3,1,0,2,2,1,1,1,1,1
1,1,3,2,2,0,2,2,0,2,3,0,0,1,2,1,0,1,2,0
2,3,0,0,1,0,3,2,0,3,3,1,0,1,1,2,0,1,1,1
3,0,3,2,2,0,3,2,2,4,2,1,0,1,2,2,0,1,1,1
4,0,1,1,1,0,2,3,0,4,0,1,0,2,2,2,0,1,2,1


In [16]:
logistic_model = LogisticRegression()

In [17]:
logistic_model = LogisticRegression()

In [18]:
logistic_model.fit(x_train, y_train)

C:\Users\ektam\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [19]:
y.head()

,result
0,1
1,2
2,1
3,1
4,2


In [20]:
logistic_model.fit(x_train, y_train.values.ravel())


LogisticRegression()

In [21]:
y_pred = logistic_model.predict(x_test)

In [22]:
accuracy_score(y_test, y_pred)

0.745